In [2]:
from mia_lib.attack.attack_reference import train_reference_models, attack_reference_mia
import torch
import os
import numpy as np
import pandas as pd
from mia_lib.config import load_config
from mia_lib.data import get_cifar10_dataloaders, create_subset_dataloader
from mia_lib.models import create_model
from mia_lib.trainer import train_model

In [3]:
config = load_config("configs/mia_config.yaml")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
trainset, testset, trainloader, testloader = get_cifar10_dataloaders(config)
print("Dataset loaded...")

Files already downloaded and verified
Files already downloaded and verified
Dataset loaded...


In [5]:
target_model = create_model(config).to(device)
os.makedirs(config["paths"]["model_save_dir"], exist_ok=True)

target_model_path = os.path.join(config["paths"]["model_save_dir"], "target_model.pth")

if os.path.exists(target_model_path):
    print(f"Target model checkpoint found at {target_model_path}. Loading...")
    target_model.load_state_dict(torch.load(target_model_path))
else:
    print(f"No target model checkoint found, Training a new one at {target_model_path}")
    # subset indices
    total_test_indices = np.arange(len(testset))
    
    # some MIA research workfloas reserve a "train" portion for shadow models. So, train model on test subset
    target_train_indices = np.random.choice(
        total_test_indices,
        config["training"]["train_subset_size"],
        replace=False
    )

    remaining_after_train = np.setdiff1d(total_test_indices, target_train_indices)
    target_eval_indices = np.random.choice(
        remaining_after_train,
        config["training"]["eval_subset_size"],
        replace=False
    )

    subset_tgt_train_loader = create_subset_dataloader(
        testset,
        target_train_indices,
        batch_size=config["training"]["train_batch_size"],
        shuffle=True,
        num_workers=config["training"]["num_workers"]
    )

    subset_tgt_eval_loader = create_subset_dataloader(
        testset,
        target_eval_indices,
        batch_size=config["training"]["eval_batch_size"],
        shuffle=False,
        num_workers=config["training"]["num_workers"]
    )

    best_acc, best_loss = train_model(
        target_model,
        subset_tgt_train_loader,
        subset_tgt_eval_loader,
        config,
        device,
        target_model_path
    )

    print(f"Target Model => Best Val Acc: {best_acc:.4f}%, Best Val Loss: {best_loss:.4f}")


Target model checkpoint found at ./models/target_model.pth. Loading...


/tmp/ipykernel_29322/1237008118.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  target_model.load_state_dict(torch.load(target_model_path))


In [6]:
reference_models = train_reference_models(config, trainset, device, n_models=5)

[Ref 0] Loaded existing indices from ./models/reference_0_indices.npz
[Ref 0] Found checkpoint at ./models/reference_model_0.pth. Loading...


/home/acer/projects/fyp/mia_lib/attack/attack_reference.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ref_model.load_state_dict(torch.load(save_path, map_location=dev

[Ref 1] Loaded existing indices from ./models/reference_1_indices.npz
[Ref 1] Found checkpoint at ./models/reference_model_1.pth. Loading...
[Ref 2] Loaded existing indices from ./models/reference_2_indices.npz
[Ref 2] Found checkpoint at ./models/reference_model_2.pth. Loading...
[Ref 3] Loaded existing indices from ./models/reference_3_indices.npz
[Ref 3] Found checkpoint at ./models/reference_model_3.pth. Loading...
[Ref 4] Loaded existing indices from ./models/reference_4_indices.npz
[Ref 4] Found checkpoint at ./models/reference_model_4.pth. Loading...


In [7]:
inference_loader = create_subset_dataloader(
    testset,
    np.arange(len(testset)),
    batch_size=config["dataset"]["eval_batch_size"],
    shuffle=False,
    num_workers=config["dataset"]["num_workers"]
)

In [8]:
df_attack_results = attack_reference_mia(
    target_model=target_model,
    reference_models=reference_models,
    loader=inference_loader, 
    device=device,
    method="mean_prob_distance",
    threshold=0.5
)

Attack R Inference: 100%|██████████| 19/19 [00:04<00:00,  3.99it/s]


In [11]:
true_labels = np.array([y for _, y in testset])
df_attack_results["true_label"] = true_labels


ValueError: Length of values (10000) does not match length of index (9728)

In [9]:
print(df_attack_results.head(10))

   sample_index  membership_prediction  distance_metric
0             0                      0         0.574487
1             1                      1         0.099009
2             2                      0         0.850441
3             3                      1         0.001411
4             4                      1         0.006863
5             5                      1         0.336123
6             6                      0         0.502248
7             7                      0         0.671788
8             8                      0         0.785682
9             9                      0         0.812469


In [10]:
# Save the results
attack_results_path = os.path.join(config["paths"]["model_save_dir"], "attack_results.csv")
df_attack_results.to_csv(attack_results_path, index=False)

# AUC
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(df_attack_results["true_label"], df_attack_results["predicted_label"])
print(f"AUC: {auc:.4f}")


KeyError: 'true_label'